## LED 閃爍控制

In [1]:
%serialconnect to --port COM5

Connecting to --port=COM5 --baud=115200 
Ready.


![LED ESP32](images/led_esp32.png)

In [2]:
from machine import Pin
import time

p0 = Pin(2, Pin.OUT)    # create output pin on GPIO2
for i in range(5):
    p0.on()                 # set pin to "on" (high) level
    time.sleep(1)
    p0.off()                # set pin to "off" (low) level
    time.sleep(1)
p0.value(1)             # set pin to on/high

..

In [3]:
# 關掉 p0
p0.value(0)

In [4]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.11-611-g7f24c2977 on 2019-12-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x2cb7b7515c8, open=True>(port='COM5', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


## LED + 按鈕控制

![LED+按鈕](images/led_sw_esp32.png)

In [ ]:
%serialconnect to --port COM5

In [ ]:
from machine import Pin

led = Pin(2, Pin.OUT)    # 設定 LED 接腳為 GPIO2 且為輸出
sw = Pin(0, Pin.IN)      # 設定 按鈕 接腳為 GPIO0 且為輸入

while True:
    val = sw.value()     # 讀取按鈕狀態
    led.value(val)

In [ ]:
%disconnect

## LED 切換開關

In [1]:
# 接線同上一個範例
%serialconnect to --port COM4

Connecting to --port=COM4 --baud=115200 
MicroPython v1.11-571-g7e374d231 on 2019-11-09; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


In [ ]:
from machine import Pin

toggle = 1 # 儲存開關狀態，預設: 高電位

led = Pin(2, Pin.OUT)    # 設定 LED 接腳為 GPIO2 且為輸出
sw = Pin(0, Pin.IN)      # 設定 按鈕 接腳為 GPIO0 且為輸入

while True:
    if sw.value == 0:    # 如果開關被按下
        toggle = not toggle # 取開關狀態變數為反向值
        led.value(toggle)   # 設定 LED 狀態
        while sw.value() == 0: # 若開關一直被按著
            pass               # 什麼事都不做

In [ ]:
# 不儲存開關狀態
from machine import Pin

led = Pin(2, Pin.OUT)    # 設定 LED 接腳為 GPIO2 且為輸出
sw = Pin(0, Pin.IN)      # 設定 按鈕 接腳為 GPIO0 且為輸入

while True:
    if sw.value == 0:    # 如果開關被按下
        led.value(not led.value())   # 設定 LED 狀態
        while sw.value() == 0: # 若開關一直被按著
            pass               # 什麼事都不做

## 解決按鈕開關彈跳問題
#### [參考](http://andrew-workshop.blogspot.com/2015/05/lab-button-debouncing.html)

![按鈕](images/Different%20views%20of%20button%20signals.png)

In [ ]:
# 解決彈跳問題
import time
from machine import Pin

led = Pin(2, Pin.OUT)    # 設定 LED 接腳為 GPIO2 且為輸出
sw = Pin(0, Pin.IN)      # 設定 按鈕 接腳為 GPIO0 且為輸入

while True:
    if sw.value == 0:    # 如果開關被按下
        time.sleep_ms(20)   # 先暫停 20ms
        led.value(not led.value())   # 設定 LED 狀態
        while sw.value() == 0: # 若開關一直被按著
            pass               # 什麼事都不做

In [ ]:
%disconnect

## AnalogRead+PWM控制LED明亮程度
#### [參考](https://www.wolfaiottw.com/pmw%E6%8E%A7%E5%88%B6led%E6%98%8E%E4%BA%AE%E7%A8%8B%E5%BA%A6/)

In [5]:
%serialconnect to --port COM5

Connecting to --port=COM5 --baud=115200 
Ready.


In [6]:
# 以腳位的 PWM 控制LED明亮度
# Complete project details at https://RandomNerdTutorials.com

from machine import Pin, PWM
from time import sleep

frequency = 5000
led = PWM(Pin(2), frequency)

while True:
  for duty_cycle in range(0, 1024):
    led.duty(duty_cycle)
    sleep(0.005)

..

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 13, in <module>
KeyboardInterrupt: 


In [7]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b'\r\nMicroPython v1.11-611-g7f24c2977 on 2019-12-02; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> '
Closing serial Serial<id=0x2cb7b80edc8, open=True>(port='COM5', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)


## 可變電阻的類比輸入
#### 如果您使用的ESP8266芯片的輸入電壓範圍為0V至1V，則需要確保A0引腳上的輸入電壓不超過1V。因此，您需要一個分壓器電路，如下所示。

![分壓電路](images/ESP-ADC-A0.png)

### 我們使用100歐姆和220歐姆的電阻，因此Vout為1V。

![分壓電路公式](images/voltage-divider-equation.png)

![32s ADC](images/nodemcu-32s-adc.png)
#### [Nodemcu 32s 教材](http://www.1zlab.com/wiki/micropython-esp32/adc/)

## 設置衰減比¶
#### 衰減器在功能上和放大器相反，放大器提供大於1的增益，衰減器提供小於1的增益。原理上衰減器由簡單的分壓電路組成。

#### ADC採樣有兩個重要的參數，一個是滿量程電壓是多少，顯然不可能超過ESP32的管腳電壓3.3v. 可以設定0-3.3v之間的一個值。

#### 巨集定義	   數值	滿量程電壓
#### ADC.ATTN_0DB	0	1.2v
#### ADC.ATTN_2_5DB	1	1.5v
#### ADC.ATTN_6DB	2	2.0v
#### ADC.ATTN_11DB	3	3.3v


## 設置解析度
#### ADC的解析度是指能夠將採集的類比信號轉化為數位信號的精度，通常我們用“位”來表述，比如8位就是指ADC可以將制定量程內的電壓信號，分別對應到0 - 2^8-1,即 0-255這256個數字上。解析度位數越高，能夠表示的也就越精確，資訊丟失的也就越少。

#### 再舉個例子，如果解析度是10位，那麼ADC採樣的取值範圍就是0 - 1023.

#### 1023 = 2^10 - 1


#### 巨集定義	數值	解析度
#### ADC.WIDTH_9BIT	0	9
#### ADC.WIDTH_10BIT	1	10
#### ADC.WIDTH_11BIT	2	11
#### ADC.WIDTH_12BIT	3	12
#### 如: adc.width(ADC.WIDTH_10BIT)

In [ ]:
%serialconnect to --port COM4

In [ ]:
from machine import ADC

adc = ADC(Pin(32))          # 在ADC引腳上創建ADC物件
adc.read()                  # 讀取測量值, 0-4095 表示電壓從 0.0v - 1.0v

adc.atten(ADC.ATTN_11DB)    # 設置 11dB 衰減輸入 (測量電壓大致從 0.0v - 3.6v)
adc.width(ADC.WIDTH_9BIT)   # 設置 9位 精度輸出 (返回值 0-511)
print(adc.read())           # 獲取重新配置後的測量值

In [ ]:
%disconnect

![PWM](images/led_p5.png)

![ESP32 VR Flow](images/esp32_vr_flow.png)

![接線圖](images/interacting-with-GPIOs-ESP32_bb.png)
#### Note: the ESP32 supports analog reading in several GPIOs: 
#### 0, 2, 4, 12, 13, 14, 15, 25, 26, 27 32, 33, 34, 35, 36, and 39.

In [ ]:
%serialconnect to --port COM4

In [ ]:
# Complete project details at https://RandomNerdTutorials.com
# Created by Rui Santos

from machine import Pin, ADC, PWM
from time import sleep

led = Pin(2, Pin.OUT)
button = Pin(15, Pin.IN)

#Configure ADC for ESP32
pot = ADC(Pin(34))
pot.width(ADC.WIDTH_10BIT)
pot.atten(ADC.ATTN_11DB)

#Configure ADC for ESP8266
#pot = ADC(0)

led_pwm = PWM(Pin(4),5000)

while True:
  button_state = button.value()
  led.value(button_state)

  pot_value = pot.read()
  led_pwm.duty(pot_value)

  sleep(0.1)

In [ ]:
%disconnect

## DAC 類比數位轉換器
### DAC是什麼¶
#### DAC 英文全稱為Digital Analog Converter，是將數位信號轉換為類比信號的轉換器。 這裡主要對比DAC與PWM，DAC輸出的是類比信號，PWM輸出的是數位信號。 DAC也可以控制LED亮度，而且亮度範圍更精細，因為DAC的輸出電壓是連續可變的，DAC控制LED也不存在PWM控制LED亮度那樣的頻閃問題，DAC可以實現比PWM更高精度的控制。另外PWM可以通過濾波器實現低精度的DAC功能。

#### DAC在專用引腳上可用，可用的DAC引腳只有兩個。 可用引腳有：GPIO25, GPIO26，輸出的電壓類比值範圍為0~3.3V


![DAC pins](images/dac-pins.png)

In [ ]:
%serialconnect to --port COM4

In [ ]:
from machine import DAC
from machine import Pin
import utime, math


dac = DAC(Pin(26,Pin.OUT), bits=12)

def pulse(dac, period, gears):
    """呼吸燈函數

    Arguments:
        dac {[DAC]} -- [DAC物件]
        period {[type]} -- [週期 ms]
        gears {[type]} -- [亮度檔位]
    """

    for i in range(2 * gears):
        dac.write(int(math.sin(i / gears * math.pi) * 2000) + 2048)
        # 延時
        utime.sleep_ms(int(period / (2 * gears)))

# 呼吸十次
for i in range(10):
    pulse(dac, 2000, 100)


In [ ]:
%disconnect